In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=cY-OysXmAZWKtlVGT-2B31DlOSn7xS8hz4qTwh1I8Y8&tc=S9R7VJU0oC7FHoBAOZsGV5mEkQZq6R6mp4WeGdSDa44&cc=P79BVaEID1DjzhDCs_X0lG65O4AI8tZCr66BsisEP7Y

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXli6eak1gDMEhJzVmHN5LuoisZ476VwHZn36bRav6T1y1EqRIipMMk

Successfully saved authorization token.


In [3]:
#Import Data for Countries
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
France = countries.filter(ee.Filter.eq('country_na','France'))

In [4]:
#Import Landsat Data
landsat8 = ee.ImageCollection(
    'LANDSAT/LC08/C01/T1_TOA')\
    .filterDate('2016-01-01', '2016-12-31')\
    .filterBounds(France)

# Get the median over time, in each band, in each pixel.
median = landsat8.median()

visualisation_parameters = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}

In [5]:
import folium

In [6]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [7]:
# Create a folium my map object.
my_map = folium.Map(location=[47.0833, 2.3986], zoom_start = 4)

# Display the median composite.
my_map.add_ee_layer(median, visualisation_parameters, 'median')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)